In [1]:
!dir
!rm -r MaskTrack_RCNN
!git clone https://github.com/umd-fire-coml/MaskTrack_RCNN

import os
from os.path import join, isfile
import re
import skimage.io
import numpy as np
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("./MaskTrack_RCNN")
sys.path.append(ROOT_DIR)  # To find local version of the library

# Import Mask RCNN
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from mrcnn import config, utils

# Root directory of the dataset
DATA_DIR = os.path.abspath("./MaskTrack_RCNN/dataset/wad")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# Local path to trained coco weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

datalab  MaskTrack_RCNN
Cloning into 'MaskTrack_RCNN'...
remote: Counting objects: 1463, done.
remote: Total 1463 (delta 0), reused 0 (delta 0), pack-reused 1463
Receiving objects: 100% (1463/1463), 763.20 MiB | 38.61 MiB/s, done.
Resolving deltas: 100% (729/729), done.
Checking out files: 100% (717/717), done.


Using TensorFlow backend.


... done downloading pretrained model!


In [2]:
!dir MaskTrack_RCNN



assets	 LICENSE	    mrcnn      requirements.txt  setup.py
dataset  MANIFEST.in	    project    samples
images	 mask_rcnn_coco.h5  README.md  setup.cfg


## Configuration

In [3]:
from project import wad_data
config = wad_data.WADConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                48
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

## Dataset

load_mask has been tested. green light .. green light

In [0]:
# Training dataset
dataset_train = wad_data.WADDataset()

dataset_train.load_data(DATA_DIR, "train")

dataset_train.prepare()

dataset_val = wad_data.WADDataset()

dataset_val.load_data(DATA_DIR, "val")

dataset_val.prepare()


In [13]:
# print("Image Count (train): {}".format(len(dataset_train.image_ids)))
# print("Class Count: {}".format(dataset_train.num_classes))

Image Count (train): 225
Class Count: 36


In [8]:
# !df -h
# !cat /proc/cpuinfo
!cat /proc/meminfo

MemTotal:       13341832 kB
MemFree:          176284 kB
MemAvailable:    6712968 kB
Buffers:           88136 kB
Cached:          6571376 kB
SwapCached:            0 kB
Active:          9080100 kB
Inactive:        3384096 kB
Active(anon):    6061952 kB
Inactive(anon):    73488 kB
Active(file):    3018148 kB
Inactive(file):  3310608 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               296 kB
Writeback:             0 kB
AnonPages:       5673660 kB
Mapped:           268840 kB
Shmem:            330744 kB
Slab:             496612 kB
SReclaimable:     461344 kB
SUnreclaim:        35268 kB
KernelStack:        3280 kB
PageTables:        23624 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6670916 kB
Committed_AS:    9746468 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
HugePages_Total: 

In [0]:
# import numpy as np
# image_ids = np.random.choice(dataset_val.image_ids, 4)

# #for image_id in image_ids:
# image_id = image_ids[0]
# image = dataset_val.load_image(image_id)
# mask, class_ids = dataset_val.load_mask(image_id)
# visualize.display_top_masks(image, mask, class_ids, dataset_val.class_names)

In [0]:
# # Compute Bounding box
# bbox = utils.extract_bboxes(mask)

# # Display image and additional stats
# print("image_id ", image_id, dataset_train.image_reference(image_id))
# log("image", image)
# log("mask", mask)
# log("class_ids", class_ids)
# log("bbox", bbox)
# # Display image and instances
# visualize.display_instances(image, bbox, mask, class_ids, dataset_train.class_names)

image_id  91 C:\Users\rmdu\MaskTrack_RCNN\dataset\wad\train_color\170908_061509080_Camera_5.jpg
image                    shape: (2710, 3384, 3)       min:    0.00000  max:  255.00000  uint8
mask                     shape: (2710, 3384, 9)       min:    0.00000  max:    1.00000  bool
class_ids                shape: (9,)                  min:    1.00000  max:    8.00000  int32
bbox                     shape: (9, 4)                min:    0.00000  max: 2809.00000  int32


## Training

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /content/MaskTrack_RCNN/logs/wad20180618T1901/mask_rcnn_wad_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


In [0]:
#model.train(dataset_train, dataset_val, 
#            learning_rate=config.LEARNING_RATE / 10,
#            epochs=1, 
#            layers="all")